## 14.3 Insertion sort

Let's apply decrease and conquer to sorting, instead of exhaustive search.

### 14.3.1 Recursive version

I start with the usual questions.

1. What are the smallest possible unsorted sequence and its sorted counterpart?
   The unsorted and sorted sequences are empty.
2. If the tail were already sorted, how could we sort the whole sequence?
   We simply insert the head in the appropriate place.

This is the core idea of **insertion sort**: the next item to be processed is
inserted into an already sorted part of the sequence.
After processing all items, the sequence is sorted.

This is one way players sort their cards: with one hand
they pick up the cards they were dealt, one by one,
inserting each one in its place in a fan of cards held in their other hand.

The algorithm for function insertion_sorted(*unsorted*, *key*) is:

1. if *unsorted* is empty:
   1. let *sorted* be ()
2. otherwise:
   1. let *subsolution* be insertion_sorted(tail(*unsorted*), *key*)
   1. let *sorted* be insert(head(*unsorted*), *subsolution*, *key*)

Inserting an item into a sorted sequence (step&nbsp;2.2) also requires
the key function.
The algorithm for function insert(*item*, *sorted*, *key*) has to compare
the keys of the item to be inserted and of the head, to know if
the item is to be inserted before or after the head.
It's another decrease-and-conquer algorithm.
I call the output variable *inserted*.

1. if *sorted* is empty:
   1. let *inserted* be (*item*)
2. otherwise if *key*(*item*) ≤ *key*(head(*sorted*)):
   1. let *inserted* be prepend(*item*, *sorted*)
2. otherwise:
   1. let *subsolution* be insert(*item*, tail(*sorted*), *key*)
   1. let *inserted* be prepend(head(*sorted*), *subsolution*)

This version of insertion sort isn't tail recursive.
It exhausts the call stack when sorting a sequence with thousands of items.
We need an iterative version.

### 14.3.2 Iterative version

Here's a version that can be implemented in-place, on the input array.

Continuing with the card game analogy,
the player fans all the dealt cards in one hand and
mentally divides them in two parts:
the sorted cards are on the left and the unsorted cards are on the right.
Initially, the sorted part is the first card and the unsorted part has all
other ones. At each step, the player uses their free hand to pick the
left-most unsorted card and put it in its correct place in the sorted part.
Each step grows the sorted part by one card and shrinks the unsorted part
by one card. The sorting ends when the unsorted part is empty.

The next table shows step by step how in-place insertion sort
alphabetically sorts a sequence of seven letters.
I use monospaced font to highlight the already sorted left part of the sequence.

0 | 1 | 2 | 3 | 4 | 5 | 6
-|-|-|-|-|-|-
`S`|O|R|T|I|N|G
`O`|`S`|R|T|I|N|G
`O`|`R`|`S`|T|I|N|G
`O`|`R`|`S`|`T`|I|N|G
`I`|`O`|`R`|`S`|`T`|N|G
`I`|`N`|`O`|`R`|`S`|`T`|G
`G`|`I`|`N`|`O`|`R`|`S`|`T`

To insert each unsorted item in the sorted part, the algorithm
shifts right all sorted items that are larger than the unsorted item.
This overwrites the unsorted item but makes place to put it in the sorted part.
This [visualisation](https://learn2.open.ac.uk/mod/oucontent/view.php?id=2554725&extra=thumbnail_idm26) shows the shifting in more detail.

I proceed directly to the code. As shown in the visualisation,
insertion sort goes through the sorted items from right to left and
shifts each one to the right if it's larger than the unsorted item.

Remember that our sorting algorithms take a [key function](../14_Sorting/14_1_sort_prep.ipynb#14.1.1-Problem)
as a second argument and that functions are objects of type `Callable`.

In [1]:
from typing import Callable


def insertion_sort(items: list, key: Callable) -> None:
    """Sort the items in-place, with keys in non-decreasing order.

    Preconditions: for any indices i and j,
    key(items[i]) and key(items[j]) are comparable
    """
    # go through all items in the unsorted part
    for first_unsorted in range(1, len(items)):
        to_sort = items[first_unsorted]
        # apply the key function given as input
        the_key = key(to_sort)
        # to start, index where to put item is index where it is now
        index = first_unsorted
        # for each sorted item larger than the one to sort
        while index > 0 and key(items[index - 1]) > the_key:
            # copy it to the right, i.e. one position up
            items[index] = items[index - 1]
            # and proceed with the next sorted item on the left
            index = index - 1
        # sorted item on the left isn't larger: we found the index
        items[index] = to_sort  # put the unsorted item there

There are [two versions](../14_Sorting/14_1_sort_prep.ipynb#14.1.1-Problem) of
the sorting problem: one modifies the input, the other returns a sorted copy.
I follow the same naming convention as Python:
`sort` for the in-place version and `sorted` for the other.

To use the `test` function, I need the sorting function to return an output,
so I will implement the `sorted` version that makes a copy of the input list,
using the `list` constructor: `copy = list(original)`.
Note that `copy = original` would create a new reference to the *same* list
and hence modify the original input.

In [2]:
from algoesup import test

%run -i ../m269_sorting


def insertion_sorted(unsorted: list, key: Callable) -> list:
    """Return a permutation with keys in non-decreasing order.

    Preconditions: for any indices i and j,
    key(unsorted[i]) and key(unsorted[j]) are comparable
    """
    result = list(unsorted)  # make a copy
    insertion_sort(result, key)
    return result


test(insertion_sorted, sorting_tests)

Testing insertion_sorted...
Tests finished: 7 passed (100%), 0 failed.


Remember that our [sorting test table](../14_Sorting/14_1_sort_prep.ipynb#14.1.2-Problem-instances)
indicates which key function (`value`, `suit` or `suit_value`) each test uses.

### 14.3.3 Complexity

For this and the following algorithms, we assume that
the key function takes constant time and so does comparing two keys.

The algorithm does the least work when the while-loop never executes.
This happens if the condition is always false:
the key of the left neighbour of the unsorted item has a lower or equal key.
In that case, the `index` variable doesn't change and the unsorted item remains
in its place, becoming the new right-most sorted item.
What's the best-case complexity of insertion sort?

___

The for-loop does *n* − 1 iterations, each only doing constant-time operations,
because the while-loop is skipped. The complexity is Θ(*n*).

The algorithm does the most work if, for each unsorted item,
*all* the sorted items are shifted.
The first unsorted item requires one shift because the sorted part has one item.
The second unsorted item requires two shifts because the sorted part has now
two items. In total, the number of shifts is
1 + 2 + ... + *n*−1 = ((*n*–1)² + (*n*–1)) / 2 by using the formula
seen in the [previous chapter](../13_Divide/13_1_decrease_one.ipynb#13.1.2-Sequence-length).
Each shift takes constant time, so the worst-case complexity is Θ(*n*²).
Can you think of a scenario with this worst-case complexity?

___

If inserting an unsorted item shifts all sorted items, then
the unsorted item must be smaller than all of them
and go to index zero. If each unsorted item triggers shifting, then it must be
smaller than the previous unsorted item. A worst-case scenario is therefore the
sequence being in descending order, or more generally, in reverse sorted order.

### 14.3.4 Performance

Let's check the best-case complexity by generating ascending
integer sequences, always doubling the length.
As previously advised, I don't start with very short sequences,
but not too long ones either because the worst-case is quadratic.

In [3]:
%run -i ../m269_sorting

for doubling in range(5):
    # generate lists of length 100, 200, 400, 800, 1600
    # each list has the integers from 0 to length - 1
    items = list(range(100 * 2**doubling))
    %timeit -r 5 insertion_sorted(items, identity)

6.95 μs ± 9.73 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)
13.7 μs ± 9.58 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)
30.4 μs ± 162 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
65 μs ± 43.8 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
135 μs ± 92.1 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)


Let's check the worst-case complexity with descending sequences.

In [4]:
for doubling in range(5):
    items = list(range(100 * 2**doubling, -1, -1))
    %timeit -r 5 insertion_sorted(items, identity)

280 μs ± 147 ns per loop (mean ± std. dev. of 5 runs, 1,000 loops each)
1.1 ms ± 806 ns per loop (mean ± std. dev. of 5 runs, 1,000 loops each)
4.74 ms ± 11.2 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
21.4 ms ± 16.1 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)
92.5 ms ± 72.6 μs per loop (mean ± std. dev. of 5 runs, 10 loops each)


Do the run-times confirm the best- and worst-case complexities?

___

For ascending sequences, the run-time doubles as the length doubles,
thus confirming the linear complexity.
For descending sequences, the run-time roughly quadruples as the length doubles,
thus confirming the quadratic complexity.

#### Exercise 14.3.1

The above code isn't just measuring the run-time of insertion sort:
it's also measuring the time to copy the input list.
Wouldn't it be better to directly call the in-place `insertion_sort`
instead of `insertion_sorted`?

_Write your answer here._

[Hint](../31_Hints/Hints_14_3_01.ipynb)
[Answer](../32_Answers/Answers_14_3_01.ipynb)

⟵ [Previous section](14_2_bogosort.ipynb) | [Up](14-introduction.ipynb) | [Next section](14_4_selection_sort.ipynb) ⟶